In [ ]:
pip install langchain_community

In [ ]:
!pip install langchain_groq

In [ ]:
!pip install faiss-cpu

In [ ]:
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.memory import ConversationBufferMemory
import os


In [ ]:
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [ ]:
def load_docs(folder_path):
  docs =[]
  for file in os.listdir(folder_path):
    if file.endswith('.txt'):
      loader = TextLoader(os.path.join(folder_path, file), encoding='utf8')
      docs.extend(loader.load())
  return docs

In [ ]:
documents = load_docs("/content/sample_data/mynotes")
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.split_documents(documents)

In [ ]:
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding)

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", k=2)


In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

rag_chain=ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True
)

In [ ]:
while True:
  query = input("You: ")
  if query.lower() in ["exit", "quit"]:
    break
  result = rag_chain.invoke({"question": query})
  print(f"Bot: {result['answer']}")

In [ ]:
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.memory import ConversationBufferMemory
import os

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

def load_docs(folder_path):
    docs = []
    for file in os.listdir(folder_path):
        if file.endswith('.txt'):
            loader = TextLoader(os.path.join(folder_path, file), encoding='utf8')
            docs.extend(loader.load())
    return docs

documents = load_docs("/content/sample_data/mynotes")

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.split_documents(documents)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embedding)

retriever = vectorstore.as_retriever(search_type="similarity", k=2)

llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    output_key="answer"  # 💡 Required fix
)


while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    result = rag_chain({"question": query})
    print(f"Bot: {result['answer']}")
